# Tutorial 2. Advanced capabilities of the MeasurementControl

In this tutorial, we will explore the more advanced features of quantify. By the end of this tutorial, we will have covered:

- Using hardware to drive experiments
- Software averaging
- Safely interrupting a running experiment

In [ ]:
import time
import random

import numpy as np
import xarray as xr
import scipy
from qcodes import ManualParameter, Parameter
%matplotlib inline
from quantify.measurement.control import MeasurementControl
import quantify.visualization.pyqt_plotmon as pqm

In [ ]:
MC = MeasurementControl('MC')
plotmon = pqm.PlotMonitor_pyqt('plotmon_MC')
MC.instr_plotmon(plotmon.name)

## A 1D hard(ware) controlled loop: Resonator Spectroscopy
### Defining a simple model

In this example, we want to find the resonance of some device. We expect to find it's resonance somewhere in the low 6GHz range, but manufacturing imperfections makes it impossible to know exactly without inspection.

We first create a `Settable` with a `QCodes.Parameter` to represent our frequency sweep range, followed by a custom `Gettable` to mock the resonating material. The Resonator will return a flat array with a spike at it's resonant frequency.

Note that the Resonator `Gettable` has a new field `soft` set to `False`. This property informs the MeasurementControl does not drive the Resonator, but instead the Resonator manages its own data acquisition.

In [ ]:
freq = ManualParameter(name='frequency', unit='Hz', label='Frequency')


class Resonator:
    def __init__(self):
        self.name = 'resonator'
        self.unit = 'V'
        self.label = 'Power'
        self.soft = False
        self.test_resonance = random.randint(6.0001e9, 6.0002e9)

    def prepare(self, setpoints):
        self.setpoints = setpoints

    def get(self):
        return (self.setpoints[:] == self.test_resonance).astype(float)


### Running the experiment
Just like our `soft` 1D loop, our complete experiment is expressed in just four lines of code.

The main difference is defining the `soft` property of our `Gettable` to False. The `MeasurementControl` will detect these settings and run in the appropriate`hard` mode.

In [ ]:
MC.set_setpars(freq)
MC.set_setpoints(np.arange(6.0001e9, 6.0002e9))
MC.set_getpars(Resonator())
dset = MC.run()

In [ ]:
plotmon.main_QtPlot

As expected, we find a single spike in the readout at the resonant frequency.

## Software Averaging: T1 Experiment
In many cases it is desirable to run an experiment many times and average the result, such as when filtering noise on instruments or measuring probability. For this purpose, the MeasurementControl provides the `soft_avg` parameter. If set to x, the experiment will run x times whilst performing a running average over each setpoint.

In this example, we want to find the relaxation time (or T1) of a Qubit. As before, we define a `Settable` and `Gettable`, representing the varying timescales we will probe through and a mock Qubit emulated in software. The mock Qubit returns the expected decay sweep but with a small amount of noise (simulating the variable qubit characteristics). We also set the qubit's T1 to 60ms - obviously in a real experiment we would be trying to discern this, but for this tutorial we set it to a known value to verify our fit later on.

In [ ]:
def decay(t, tau):
    return np.exp(-t/tau)

time_par = ManualParameter(name='time', unit='s', label='Measurement Time')

class MockQubit:
    def __init__(self):
        self.name = 'qubit'
        self.unit = '%'
        self.label = 'High V'
        self.soft = False
        self.setpoints = []
        self.test_relaxation_time = 60e-6

    def prepare(self, setpoints):
        self.setpoints = setpoints

    def get(self):
        return np.array(list(map(lambda x: decay(x, self.test_relaxation_time) + random.uniform(-0.1, 0.1), self.setpoints)))


We will then sweep through 0 to 300ms, getting our data from the mock Qubit:

In [ ]:
MC.set_setpars(time_par)
MC.set_setpoints(np.linspace(0.0, 300.0e-6, 300))
MC.set_getpars(MockQubit())
MC.run('noisy')
plotmon.main_QtPlot

Alas, the noise in the signal has made this result unusable! Let's set the `soft_avg` parameter of the MeasurementControl to 5000. As above, this will run the experiment 5000 times, averaging the results and hopefully filtering out the noise.

In [ ]:
MC.soft_avg(5000)
dset = MC.run('averaged')
plotmon.main_QtPlot

Success! We now have a smooth decay curve based on the characteristics of our qubit. All that remains is to run a fit against the expected values and we can solve for T1.

In [ ]:
from lmfit import Model

model = Model(decay, independent_vars=['t'])
fit_res = model.fit(dset['y0'].values, t=dset['x0'].values, tau=1)

fit_res.plot_fit(show_init=True)
fit_res.values